In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import os, cv2, numpy as np

In [2]:
import os
import cv2
import numpy as np
import torch
import torch.utils.data as data

def dataloading(image_folder):
    image_path = os.path.join(os.getcwd(), image_folder)
    image_list = []
    label_list = []

    for path in os.listdir(image_path):  # 파일명 예: Train_aug_num_label.png
        # 흑백 이미지 로드 (H, W)
        img = cv2.imread(os.path.join(image_path, path), cv2.IMREAD_GRAYSCALE)

        # 이미지 크기 확인 후 32x32로 리사이징 (필요 시)
        img = cv2.resize(img, (32, 32)) if img.shape != (32, 32) else img

        # 채널 차원 추가 (1, 32, 32) → CNN 입력 형태 맞추기
        img = np.expand_dims(img, axis=0)  # (1, H, W)

        # 정규화 (0~1 범위)
        img = img.astype(np.float32) / 255.0  

        # 라벨 추출 (one-hot 대신 정수형으로 변경)
        class_num = int(path.split('_')[-1].split('.')[0])  # 파일명에서 클래스 인덱스 추출

        # 리스트에 추가
        image_list.append(img)
        label_list.append(class_num)

    # NumPy → PyTorch Tensor 변환
    image_tensor = torch.tensor(np.array(image_list), dtype=torch.float32)  # (B, 1, 32, 32)
    label_tensor = torch.tensor(np.array(label_list), dtype=torch.long)  # 정수형 라벨 (B,)

    # PyTorch Dataset & DataLoader 생성
    dataset = data.TensorDataset(image_tensor, label_tensor)
    dataloader = data.DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)  # num_workers 조정 가능

    return dataloader


In [4]:
from models.model1.model import Net
train_model = Net()
dataloader = dataloading('data/train_data')


In [5]:
import torch.nn as nn
import torch.optim as optim
import os

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(train_model.parameters(), lr=0.001)
train_model.train()

# Training parameters
num_epochs = 100
total_step = len(dataloader)

# 저장할 디렉토리 생성 (필요시)
save_dir = "models/model1"
os.makedirs(save_dir, exist_ok=True)

# Best model tracking
best_accuracy = 0.0  # 최고 성능 저장

# Train loop
for epoch in range(num_epochs):
    correct = 0  # Accuracy 계산을 위한 변수 초기화
    total = 0
    epoch_loss = 0  # Epoch별 Loss 저장

    for i, (images, labels) in enumerate(dataloader):
        images = images.float()  # 이미지 float32 변환
        labels = labels.long()   # 라벨을 long (int64) 타입으로 변환
        
        # Forward
        outputs = train_model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        # Backward & Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy 계산
        _, predicted = torch.max(outputs, 1)  # 예측값
        correct += (predicted == labels).sum().item()  # 맞춘 개수
        total += labels.size(0)  # 전체 개수
        
        # Loss 출력 (100 step마다)
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}")

    # Epoch 끝난 후 정확도 출력
    accuracy = 100 * correct / total
    avg_loss = epoch_loss / total_step
    print(f"Epoch [{epoch+1}/{num_epochs}] Finished - Accuracy: {accuracy:.2f}%, Avg Loss: {avg_loss:.4f}")

    # 🔹 Best Model 저장 (Accuracy 최고값 갱신 시)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_path = os.path.join(save_dir, "best.pth")
        torch.save(train_model.state_dict(), best_model_path)
        print(f"✅ Best Model Saved! Accuracy: {best_accuracy:.2f}%")

    # 🔹 Last Model 저장 (항상 저장)
    last_model_path = os.path.join(save_dir, "last.pth")
    torch.save(train_model.state_dict(), last_model_path)

print("🎉 Training Finished!")
print(f"🔥 Best Model Accuracy: {best_accuracy:.2f}% (Saved as 'best.pth')")
print(f"📌 Last Model Saved as 'last.pth'")


Epoch [1/100], Step [100/128], Loss: 1.9512
Epoch [1/100] Finished - Accuracy: 18.00%, Avg Loss: 2.1389
✅ Best Model Saved! Accuracy: 18.00%
Epoch [2/100], Step [100/128], Loss: 1.8440
Epoch [2/100] Finished - Accuracy: 33.27%, Avg Loss: 1.8455
✅ Best Model Saved! Accuracy: 33.27%
Epoch [3/100], Step [100/128], Loss: 2.5936
Epoch [3/100] Finished - Accuracy: 49.32%, Avg Loss: 1.5548
✅ Best Model Saved! Accuracy: 49.32%
Epoch [4/100], Step [100/128], Loss: 0.8484
Epoch [4/100] Finished - Accuracy: 58.32%, Avg Loss: 1.2192
✅ Best Model Saved! Accuracy: 58.32%
Epoch [5/100], Step [100/128], Loss: 1.2679
Epoch [5/100] Finished - Accuracy: 63.99%, Avg Loss: 1.0533
✅ Best Model Saved! Accuracy: 63.99%
Epoch [6/100], Step [100/128], Loss: 0.2139
Epoch [6/100] Finished - Accuracy: 68.69%, Avg Loss: 0.8881
✅ Best Model Saved! Accuracy: 68.69%
Epoch [7/100], Step [100/128], Loss: 1.4842
Epoch [7/100] Finished - Accuracy: 73.78%, Avg Loss: 0.8201
✅ Best Model Saved! Accuracy: 73.78%
Epoch [8/100]

In [18]:
def predict_total(data_folder,models,encoder:dict):
    # dict 형태: 'cricles': 0, 'triangles': 1, 'squares': 2
    images = []
    encoder = {v: k for k, v in encoder.items()} 
    for i in range(0,len(os.listdir(data_folder))):
        num = str(i).zfill(3)
        path = f'TEST_{num}_0.png'
        images.append(cv2.imread(os.path.join(data_folder, path), cv2.IMREAD_GRAYSCALE))
    predicted = []
    for i, image in enumerate(images):
        image = cv2.resize(image, (32, 32)) if image.shape != (32, 32) else image
        image = np.expand_dims(image, axis=0)
        image = torch.tensor(image, dtype=torch.float32) / 255.0
        image = image.unsqueeze(0)
        output = models(image)
        _, pred = torch.max(output, 1) # 예측값
        pred = encoder.get(int(pred))
        print(f"Image {i} Prediction: {pred}")
        predicted.append(pred)
    return predicted

In [19]:
import json
encoder = json.load(open('data/label_encode.json'))
model = Net()
model.load_state_dict(torch.load('models/model1/best.pth'))
model.eval()
data_folder = 'data/test'
predicted = predict_total(data_folder,model,encoder)


Image 0 Prediction: airplane
Image 1 Prediction: emotion_face
Image 2 Prediction: emotion_face
Image 3 Prediction: police_car
Image 4 Prediction: truck
Image 5 Prediction: airplane
Image 6 Prediction: emotion_face
Image 7 Prediction: cat
Image 8 Prediction: ball
Image 9 Prediction: police_car
Image 10 Prediction: ball
Image 11 Prediction: apple
Image 12 Prediction: truck
Image 13 Prediction: ball
Image 14 Prediction: truck
Image 15 Prediction: ball
Image 16 Prediction: airplane
Image 17 Prediction: police_car
Image 18 Prediction: cat
Image 19 Prediction: emotion_face
Image 20 Prediction: truck
Image 21 Prediction: truck
Image 22 Prediction: apple
Image 23 Prediction: truck
Image 24 Prediction: airplane
Image 25 Prediction: emotion_face
Image 26 Prediction: police_car
Image 27 Prediction: bird
Image 28 Prediction: emotion_face
Image 29 Prediction: cat
Image 30 Prediction: cat
Image 31 Prediction: truck
Image 32 Prediction: emotion_face
Image 33 Prediction: apple
Image 34 Prediction: air

In [20]:
import pandas as pd
df = pd.read_csv('data/sample_submission.csv')
df['label'] = predicted
df.to_csv('submission.csv', index=False)

In [21]:
df

,ID,label
0,TEST_000,airplane
1,TEST_001,emotion_face
2,TEST_002,emotion_face
3,TEST_003,police_car
4,TEST_004,truck
...,...,...
245,TEST_245,police_car
246,TEST_246,cat
247,TEST_247,cat
248,TEST_248,police_car
